In [6]:
# Generic OpenAI-compatible endpoint using ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage
import os
default_model_name = os.environ["OPENAI_MODEL"]
llm = ChatOpenAI(model=default_model_name)


In [2]:
messages = [HumanMessage("What is the capital of France?")]
llm.invoke(messages) 

AIMessage(content='The capital of France is **Paris**.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 27, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 7}, 'words': 52}, 'model_name': 'zai', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f47a941d-fd4f-4932-83c5-4de600cc733c-0')

In [3]:
system_msg = SystemMessage(
    '''You are a helpful assistant that responds to questions with three
        exclamation marks.'''
)
messages = [system_msg,HumanMessage("What is the capital of France?")]
llm.invoke(messages) 

AIMessage(content='The capital of France is Paris!!!', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 32, 'total_tokens': 44, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 4}, 'words': 50}, 'model_name': 'zai', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-16b4fee3-44e8-4b66-ae02-9a95642c0913-0')

In [9]:
context="""Kemajuan terbaru dalam NLP didorong oleh Large Language Models
(LLM). Model-model ini mengungguli model-model yang lebih kecil dan telah
menjadi sangat berharga bagi para pengembang yang membuat aplikasi dengan
kemampuan NLP. Pengembang dapat mengakses model-model ini melalui library
`transformers` dari Hugging Face, atau dengan memanfaatkan layanan dari OpenAI
dan Cohere melalui library `openai` dan `cohere`."""
question="Penyedia model mana saja yang menawarkan LLM?"


In [11]:
from langchain_core.prompts import ChatPromptTemplate
template = ChatPromptTemplate.from_messages([
    ('system', '''Answer the question based on the context below. If the
        question cannot be answered using the information provided, answer with
        "I don\'t know".'''),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}'),
])
chain = template | llm
result=chain.invoke({
    "context":context,
    "question":question
})
print(result.content)

Berdasarkan konteks yang diberikan, penyedia model yang menawarkan LLM antara lain:

1. **Hugging Face** (melalui library `transformers`)
2. **OpenAI** (melalui library `openai`)
3. **Cohere** (melalui library `cohere`)


In [12]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("""Jawab pertanyaan berdasarkan konteks
    di bawah ini. Jika pertanyaan tidak dapat dijawab menggunakan informasi
    yang diberikan, jawab dengan "Saya tidak tahu".

Konteks: {context}

Pertanyaan: {question}

Jawaban: """)

result=chain.invoke({
    "context":context,
    "question":question
})
print(result.content)

Berdasarkan konteks yang diberikan, penyedia model yang menawarkan LLM antara lain:

1. **Hugging Face** - melalui library `transformers`.
2. **OpenAI** - melalui layanan dan library `openai`.
3. **Cohere** - melalui layanan dan library `cohere`.


In [3]:
from langchain_core.pydantic_v1 import BaseModel

class JawabanDenganJustifikasi(BaseModel):
    '''Jawaban atas pertanyaan pengguna beserta justifikasi untuk jawaban tersebut.'''
    jawaban: str
    '''Jawaban atas pertanyaan pengguna'''
    justifikasi: str
    '''Justifikasi untuk jawaban'''

structured_llm = llm.with_structured_output(JawabanDenganJustifikasi)

structured_llm.invoke("""Mana yang lebih berat, satu pon batu bata atau satu pon bulu?""")

JawabanDenganJustifikasi(jawaban='Keduanya sama berat.', justifikasi='Satu pon batu bata dan satu pon bulu memiliki berat yang sama karena satuan "pon" menyatakan berat yang identik, terlepas dari jenis materialnya. Meskipun volume bulu jauh lebih besar daripada batu bata, berat keduanya tetap sama yaitu satu pon.')